In [ ]:
!pip install pyspark

In [ ]:
# importar bibliotecas
import pyspark
from pyspark.sql import SparkSession

#Criação de uma sessão Spark(caso ela não exista) ou recuperação de uma 
#sessão existente com o mesmo nome
spark = SparkSession.builder.appName("Censo Escolar").master('local[*]').getOrCreate()

# criação do contexto

sc = spark.sparkContext

rdd = sc.textFile("censo_escolar_2021.csv")
rdd.take(4)

In [ ]:
# Ignorar o Cabeçalho

rdd = rdd.filter(lambda x:x.split(";")[0]!="NU_ANO_CENSO")
rdd.collect()

## **Resolução Questão 1**

In [28]:
# Questão 1 (Primeira Maneira para realizar a Questão - Mais elaborada)

rdd_map = rdd.map(lambda x: (x.split(";")[6], 1))
rdd_map.take(4)

[("Alta Floresta D'Oeste", 1),
 ("Alta Floresta D'Oeste", 1),
 ("Alta Floresta D'Oeste", 1),
 ("Alta Floresta D'Oeste", 1)]

In [29]:
rdd_reduce = rdd_map.reduceByKey(lambda x,y : x+y)
rdd_reduce.take(4)

[('Colorado do Oeste', 15),
 ('Costa Marques', 15),
 ("Nova Brasil�ndia D'Oeste", 11),
 ("Alvorada D'Oeste", 20)]

In [30]:
rdd_curitiba = rdd_reduce.filter(lambda x: x[0] == 'Curitiba')
rdd_curitiba.take(5)

[('Curitiba', 1181)]

In [31]:
rdd_curitiba.coalesce(1).saveAsTextFile("Escolas-Curitiba.txt")

In [ ]:
# Questão 1 (Segunda Maneira para realizar a Questão - Mais simplificada)

rddAlternativo = rdd.filter(lambda x:x.split(";")[6] == "Curitiba")
rddAlternativo.take(4)

In [33]:
rddAlternativo.count()

1181

## **Resolução Questão 2**

In [34]:
rdd_rg_map = rdd.map(lambda x:(x.split(";")[1], 1))
rdd_rg_map.take(10)

[('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1)]

In [35]:
rdd_rg_reduce = rdd_rg_map.reduceByKey(lambda x,y : x+y)
rdd_rg_reduce.collect()

[('Norte', 26095),
 ('Sul', 29018),
 ('Centro-Oeste', 11659),
 ('Nordeste', 79039),
 ('Sudeste', 75329)]

In [36]:
rdd_rg_reduce.coalesce(1).saveAsTextFile("Numero-Escolas-Região_Geográfica.txt")

## **Resolução Questão 4**

In [37]:
pairRDD_media = rdd.map(lambda x:(x.split(";")[1], (int(x.split(";")[305]), 1)))
pairRDD_media.take(6)

[('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (327, 1))]

In [ ]:
pairRDD_soma_q3 = pairRDD_media.reduceByKey(lambda x,y: x+y)
pairRDD_soma_q3.take(6)

## **Resolução Questão 5**

In [38]:
#Validar e ignorar o campos que estão em branco no arquivo
rdd_filter = rdd.filter(lambda x:x.split(";")[17] != " " and x.split(";")[15] != " ")

#PairRdd que realiza um Map que retorna uma nova lista com os valores dos índices (Localização e Dependência)
pairRdd_values = rdd.map(lambda x: ((x.split(";")[17], x.split(";")[15]), 1))
pairRdd_values.take(10)

[(('2', '2'), 1),
 (('1', '2'), 1),
 (('2', '3'), 1),
 (('1', '3'), 1),
 (('2', '3'), 1),
 (('1', '2'), 1),
 (('1', '3'), 1),
 (('2', '3'), 1),
 (('1', '2'), 1),
 (('2', '3'), 1)]

In [39]:
#somar as ocorrencias das chaves, apresentando a quantidade de escolas. 
#Por tipo de localização e Dependência
contagem_dependencia_local = pairRdd_values.reduceByKey(lambda x,y: x+y)
contagem_dependencia_local.collect()

[(('1', '1'), 619),
 (('1', '2'), 26796),
 (('1', '3'), 64278),
 (('2', '4'), 934),
 (('2', '2'), 6768),
 (('2', '3'), 70090),
 (('1', '4'), 51559),
 (('3', '0'), 1),
 (('2', '1'), 95)]

In [40]:
#Criação do arquivo TXT com o resultado da função
contagem_dependencia_local.coalesce(1).saveAsTextFile("Escolas_local_Dependencia.txt")

## **Resolução Questão 6**

In [ ]:
rdd_filtro = rdd.filter(lambda x:x.split(";")[4] != " " and x.split(";")[15] != " ")

pairRDD_quant = rdd_filtro.map(lambda x: ((x.split(";")[4], x.split(";")[15]), 1))
pairRDD_quant.collect()

In [ ]:
rdd_somaQuant = pairRDD_quant.reduceByKey(lambda x,y: x+y)
rdd_somaQuant.collect()

In [45]:
rdd_somaQuant_ordenado = rdd_somaQuant.sortBy(lambda x:x[1] , ascending=False)
rdd_somaQuant_ordenado.collect()

[(('BA', '3'), 15659),
 (('SP', '3'), 13822),
 (('SP', '4'), 13550),
 (('MG', '3'), 13363),
 (('MA', '3'), 11432),
 (('PA', '3'), 10371),
 (('CE', '3'), 7255),
 (('SP', '2'), 6572),
 (('RJ', '3'), 6368),
 (('MG', '4'), 6259),
 (('PE', '3'), 6184),
 (('RS', '3'), 5867),
 (('PR', '3'), 5487),
 (('RJ', '4'), 5301),
 (('PI', '3'), 5281),
 (('AM', '3'), 5080),
 (('MG', '2'), 4319),
 (('SC', '3'), 4290),
 (('PB', '3'), 3895),
 (('RN', '3'), 3435),
 (('BA', '4'), 3297),
 (('PE', '4'), 3216),
 (('RS', '4'), 3165),
 (('ES', '3'), 2752),
 (('GO', '3'), 2751),
 (('CE', '4'), 2604),
 (('RS', '2'), 2549),
 (('AL', '3'), 2474),
 (('PR', '4'), 2399),
 (('PR', '2'), 2164),
 (('MT', '3'), 1823),
 (('SC', '4'), 1608),
 (('SE', '3'), 1572),
 (('RJ', '2'), 1547),
 (('TO', '3'), 1534),
 (('MA', '4'), 1394),
 (('SC', '2'), 1371),
 (('GO', '4'), 1334),
 (('MA', '2'), 1305),
 (('BA', '2'), 1168),
 (('PE', '2'), 1083),
 (('PA', '4'), 1046),
 (('GO', '2'), 1040),
 (('PB', '4'), 1022),
 (('AC', '3'), 1009),
 (('

In [44]:
rdd_somaQuant_ordenado.coalesce(1).saveAsTextFile("Quant-de-escolas-por-UF-e-DependenciaAdmin.txt")